In [ ]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import multiprocessing as mp
import glob
import sys, os
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm
tqdm.pandas()
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import jieba

In [ ]:
#pip install numba --upgrade  

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec

class MonitorCallback(CallbackAny2Vec):
  def __init__(self, name):
    self.name = name
    self.epoch = 1
    self.timer = gezi.Timer()
    
  def on_epoch_end(self, model):
    # TODO 为什么打印train loss一直是0
    print('name:', self.name, 'epoch:', self.epoch, "model loss:", model.get_latest_training_loss(), f'elapsed minutes: {self.timer.elapsed_minutes():.2f}')  # print loss
    self.epoch += 1

In [ ]:
vocab_names = [
                'tag',
                'word',
              ]
vocabs = {}
for vocab_name in vocab_names:
  vocab_file =  f'../input/{vocab_name}_vocab.txt'
  if vocab_name != 'word':
    vocab = gezi.Vocab(vocab_file)
  else:
    vocab = gezi.Vocab(vocab_file, 200)
  vocabs[vocab_name] = vocab


In [ ]:
vocabs['tag']

In [ ]:
d = pd.read_csv('../input/info/infos.csv')

In [ ]:
d.head(30)

In [ ]:
def gen_w2v(window=32, min_count=1, emb_dim=256):
  sentences = []
  for row in tqdm(d.itertuples(), total=len(d)):
    row = row._asdict()
    l = [row['category_id'], '[SEP1]', row['cat'], '[SEP2]', *(str(row['tag_id']).split(',')), '[CLS]', *jieba.cut(str(row['title'])), '[SEP]', *jieba.cut(str(row['asr_text'])), '[SEP]']
    sentences.append(l)
  ic(len(sentences))
  name = 'word'
  monitor = gezi.MonitorCallback(name) 
  w2v = Word2Vec(sentences, vector_size=emb_dim, window=window, min_count=min_count, sg=1, workers=mp.cpu_count(), epochs=10, callbacks=[monitor])
  ofile = f'../input/w2v/jieba/{emb_dim}/{name}.pkl'
  gezi.try_mkdir(os.path.dirname(ofile))
  for name in vocabs:
    vocab = vocabs[name]
    # emb = np.zeros([vocab.size(), emb_dim])
    emb = np.random.uniform(-0.05, 0.05,(vocab.size(), emb_dim))
    for i in range(vocab.size()):
      word = vocab.key(i) 
      if word in w2v.wv:
        emb[i] = w2v.wv[word]
    ofile = f'../input/w2v/jieba/{emb_dim}/{name}.npy'
    np.save(ofile, emb)
  
  return w2v

In [ ]:
gen_w2v(emb_dim=256)

In [ ]:
gen_w2v(emb_dim=512)

In [ ]:
emb_dims = [256, 512]
for emb_dim in emb_dims:
  w2v = gezi.read_pickle( f'../input/w2v/jieba/{emb_dim}/word.pkl')
  for name in vocabs:
    print(name)
    vocab = vocabs[name]
    # emb = np.zeros([vocab.size(), emb_dim])
    emb = np.random.uniform(-0.05, 0.05,(vocab.size(), emb_dim))
    for i in range(vocab.size()):
      word = vocab.key(i) 
      if word in w2v.wv:
        emb[i] = w2v.wv[word]
    ofile = f'../input/w2v/jieba/{emb_dim}/{name}.npy'
    np.save(ofile, emb)
    print(emb)
  